In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [38]:
questions = pd.DataFrame.from_csv('question_simple.csv', index_col=None)
questions.head()

,QuestionId,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate,QScore,QVotes
0,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,12,12
1,2,2011-11-15T20:50:11.270,2011-12-02T11:38:48.620,10,2011-11-16T00:00:00.000,12,12
2,3,2011-11-15T20:52:27.287,2015-07-18T09:49:54.953,109,2013-01-19T00:00:00.000,26,30
3,7,2011-11-15T20:56:40.383,2015-11-23T12:25:29.887,2977,2012-10-22T00:00:00.000,23,26
4,14,2011-11-15T21:01:05.117,2015-08-13T13:14:56.870,20,2011-11-16T00:00:00.000,11,12


In [39]:
a_questions = pd.DataFrame.from_csv('question_votes.csv', index_col=None)
a_questions[a_questions.AcceptedAnsId.notnull()].head()

,QuestionId,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate,VoteType,QVoteCreation
0,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037
1,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037
2,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037
3,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037
4,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037


In [40]:
get_votes_qv = lambda df: pd.Series((df.VoteType==2).cumsum() + (df.VoteType==3).cumsum(),name='QVotes')
get_score_qv = lambda df: pd.Series((df.VoteType==2).cumsum() - (df.VoteType==3).cumsum(),name='QScore')

predictors_qvotes = ['QuestionId','QuestionCreation','QuestionLastActivity','AcceptedAnsId','AcceptedDate','QVoteCreation']
f_q = lambda df: pd.concat([df[cname] for cname in df.columns.values.tolist() if cname in predictors_qvotes]+[get_score_qv(df),get_votes_qv(df)],axis=1)
a_questions = a_questions.sort_values(by='QVoteCreation').groupby(['QuestionId']).apply(f_q)
a_questions.head(10)

,QuestionId,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate,QVoteCreation,QScore,QVotes
0,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,1,1
11,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,2,2
10,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,3,3
9,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,4,4
7,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,5,5
6,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,6,6
8,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,7,7
4,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,8,8
3,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,9,9
2,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,10,10


In [41]:
a_votes = pd.DataFrame.from_csv('votes-answers.csv', index_col=None)
a_votes = pd.merge(a_votes, a_questions, how='inner', on=['QuestionId'],suffixes=['_v', '_q'])
a_votes.head()

,VoteId,VoteCreation,AnsCreation,VoteType,AnsId,QuestionId,AnsWordCount,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate,QVoteCreation,QScore,QVotes
0,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,1,1
1,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,2,2
2,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,3,3
3,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,4,4
4,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,5,5


In [42]:
predictors_raw_votans =['VoteId','VoteCreation','AnsCreation','VoteType','AnsId','QuestionId','AnsWordCount','QuestionCreation','AcceptedAnsId','AcceptedDate']

valid_qavotes = lambda df: df[df.VoteCreation>=df.QVoteCreation]
#Use twice valid_qavotes, could use once to improve efficiency, but check correctness of index selection
get_max_qv = lambda df: valid_qavotes(df).loc[valid_qavotes(df).QVotes.idxmax(),['QScore','QVotes']].squeeze()
get_latest_qv = lambda df : pd.Series([0,0],index=['QScore','QVotes']) if not (df.VoteCreation>=df.QVoteCreation).any() else get_max_qv(df)
get_head = lambda df: [df[cname].iloc[0] for cname in df.columns.values.tolist() if cname in predictors_raw_votans]
get_qv = lambda df : pd.Series(get_head(df),index=predictors_raw_votans).append(get_latest_qv(df)).to_frame()

a_votes = a_votes.sort_values(by='VoteCreation').groupby(['VoteId']).apply(get_qv).unstack(level=-1).reset_index(level=[0],drop=True)
a_votes.drop(a_votes.columns[[0]], axis=1, inplace=True)
a_votes.columns = a_votes.columns.droplevel()
a_votes.head()

,VoteCreation,AnsCreation,VoteType,AnsId,QuestionId,AnsWordCount,QuestionCreation,AcceptedAnsId,AcceptedDate,QScore,QVotes
0,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0
1,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0
2,2011-11-15T00:00:00.000,2011-11-15T20:54:09.083,2,5,1,37,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0
3,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,3,4,1,190,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0
4,2011-11-15T00:00:00.000,2011-11-15T20:54:49.410,2,6,2,51,2011-11-15T20:50:11.270,10,2011-11-16T00:00:00.000,0,0


In [43]:
a_votes['AcceptedDate'].fillna(pd.to_datetime('20160320'),inplace=True) #Date After Data Set Collection
a_votes['AcceptedAge'] = (pd.to_datetime(a_votes.AcceptedDate,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_votes.QuestionCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_votes['AcceptedAge'] = a_votes['AcceptedAge'] + 1

a_votes.loc[a_votes.AcceptedDate == pd.to_datetime('20160320'), 'AcceptedAge'] = -1
a_votes['Age'] = (pd.to_datetime(a_votes.VoteCreation,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_votes.QuestionCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_votes['Age'] = a_votes['Age'] + 1
a_votes.head()

,VoteCreation,AnsCreation,VoteType,AnsId,QuestionId,AnsWordCount,QuestionCreation,AcceptedAnsId,AcceptedDate,QScore,QVotes,AcceptedAge,Age
0,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0,6,0
1,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0,6,0
2,2011-11-15T00:00:00.000,2011-11-15T20:54:09.083,2,5,1,37,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0,6,0
3,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,3,4,1,190,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0,6,0
4,2011-11-15T00:00:00.000,2011-11-15T20:54:49.410,2,6,2,51,2011-11-15T20:50:11.270,10,2011-11-16T00:00:00.000,0,0,1,0


In [44]:
a_votes.drop(a_votes.columns[[0, 1, 6, 8]], axis=1, inplace=True)

In [34]:
#a_comments = pd.DataFrame.from_csv('comment-ans.csv', index_col=None)
#a_comments = pd.merge(a_comments, questions, how='inner', on=['QuestionId'],suffixes=['_vot', '_quest'])

#a_comments['Age'] = (pd.to_datetime(a_comments.CommentCreation,format='%Y-%m-%d %H:%M:%S.%f')
                 # -pd.to_datetime(a_comments.QuestionCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
#a_comments['Age'] = a_comments['Age'] + 1
#a_comments = a_comments[['AnsId','Age']]
#a_comments.head(10)

,AnsId,Age
0,4,1
1,4,1
2,5,1
3,56,1
4,56,1
5,56,1
6,56,1
7,7030,951
8,7030,1285
9,15104,1436


## Each entry in the following tables represents the total votes in an answer in a given day (and associated attributes for the answer and question)

In [45]:
get_score = lambda df: sum(df.VoteType==2) - sum(df.VoteType==3)
get_votes = lambda df: sum(df.VoteType==2) + sum(df.VoteType==3)

predictors = ['QuestionId','AnsWordCount','AcceptedAnsId','AcceptedAge','QScore',
              'QVotes','Score','Votes','Upvotes','Downvotes']
f = lambda df: pd.Series([df.QuestionId.iloc[0],df.AnsWordCount.iloc[0],df.AcceptedAnsId.iloc[0],df.AcceptedAge.iloc[0],
                          df.QScore.iloc[0],df.QVotes.iloc[0],get_score(df),get_votes(df),sum(df.VoteType==2),sum(df.VoteType==3)],index = predictors)
a_groups = a_votes.sort_values(by='Age').groupby(['AnsId','Age']).apply(f)
a_groups = a_groups.reset_index(level=[0,1],drop=False)
a_groups.head()

,AnsId,Age,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,Upvotes,Downvotes
0,4,0,1,190,56,6,0,0,1,3,2,1
1,4,1436,1,190,56,6,12,12,1,1,1,0
2,5,0,1,37,56,6,0,0,1,1,1,0
3,5,108,1,37,56,6,12,12,-1,1,0,1
4,6,0,2,51,10,1,0,0,1,1,1,0


In [46]:
cum_votes = lambda df: pd.Series(df['Votes'].cumsum(),name='CumVotes')
cum_score = lambda df: pd.Series(df['Score'].cumsum(),name='CumScore')

get_cumulative =lambda df: pd.concat([df[cname] for cname in df.columns.values.tolist()] + [cum_votes(df),cum_score(df)],axis=1)
ff = lambda df: get_cumulative(df.sort_values(by='Age'))
a_groups_c = a_groups.groupby(['AnsId']).apply(ff)
a_groups_c.head()

AnsId   Age  QuestionId  AnsWordCount  AcceptedAnsId  AcceptedAge  \
AnsId                                                                        
4     0      4     0           1           190             56            6   
      1      4  1436           1           190             56            6   
5     2      5     0           1            37             56            6   
      3      5   108           1            37             56            6   
6     4      6     0           2            51             10            1   

         QScore  QVotes  Score  Votes  Upvotes  Downvotes  CumVotes  CumScore  
AnsId                                                                          
4     0       0       0      1      3        2          1         3         1  
      1      12      12      1      1        1          0         4         2  
5     2       0       0      1      1        1          0         1         1  
      3      12      12     -1      1        0          1         2         0  
6     4       0       0      1      1        1          0         1         1

In [47]:
## SHOULD RETURN 0,3
tt = a_groups_c.groupby(['AnsId','Age']).Votes.count().reset_index(level=[0,1],drop=False)
tt[tt.Votes>1].shape

(0, 3)

In [48]:
prior_quality = float(a_groups_c['Upvotes'].sum())/(a_groups_c['Upvotes'].sum() + a_groups_c['Downvotes'].sum())

In [49]:
a_groups_c['ReScore'] = (a_groups_c['CumScore']+prior_quality)/(a_groups_c['CumVotes']+1.0)
a_groups_c['QReScore'] = a_groups_c['QScore']/(a_groups_c['QVotes']+1.0)
a_groups_c.head()

AnsId   Age  QuestionId  AnsWordCount  AcceptedAnsId  AcceptedAge  \
AnsId                                                                        
4     0      4     0           1           190             56            6   
      1      4  1436           1           190             56            6   
5     2      5     0           1            37             56            6   
      3      5   108           1            37             56            6   
6     4      6     0           2            51             10            1   

         QScore  QVotes  Score  Votes  Upvotes  Downvotes  CumVotes  CumScore  \
AnsId                                                                           
4     0       0       0      1      3        2          1         3         1   
      1      12      12      1      1        1          0         4         2   
5     2       0       0      1      1        1          0         1         1   
      3      12      12     -1      1        0          1         2         0   
6     4       0       0      1      1        1          0         1         1   

          ReScore  QReScore  
AnsId                        
4     0  0.490730  0.000000  
      1  0.592584  0.923077  
5     2  0.981460  0.000000  
      3  0.320974  0.923077  
6     4  0.981460  0.000000

### Careful in this cell!!! It introduces duplicate AnsId-pairs (ignore for now)

In [55]:
#votes_com = pd.merge(a_groups_c, a_comments, how='left', on=['AnsId'],suffixes=['_vot', '_com'])
#def shrink_comments(df):
#    dd = {cname:[df[cname].iloc[0]] for cname in df.columns.values.tolist() if cname not in ['AnsId','Age_vot'] }
#    dd.update({'Comments':[sum(df['Age_vot']>=df['Age_com'])]})
#    return pd.DataFrame(dd)
#votes_com_f = votes_com.groupby(['AnsId','Age_vot']).apply(shrink_comments).reset_index(level=[0,1],drop=False)

In [56]:
## SHOULD RETURN 0,3
#tt = votes_com_f.groupby(['AnsId','Age_vot']).Votes.count().reset_index(level=[0,1],drop=False)
#tt[tt.Votes>1].shape

(0, 3)

## First Time series cleaned (model for the number of votes)

# Check inconsistent entries

In [59]:
votes_com_f=a_groups_c

In [61]:
votes_com_f[(votes_com_f['AnsId'].isnull()) | (votes_com_f['Age'].isnull())
            | (votes_com_f['Votes'].isnull()) | (votes_com_f['Score'].isnull())]

,,AnsId,Age,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,Upvotes,Downvotes,CumVotes,CumScore,ReScore,QReScore
AnsId,,,,,,,,,,,,,,,,,


In [62]:
votes_com_f[votes_com_f.Age < 0]

,,AnsId,Age,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,Upvotes,Downvotes,CumVotes,CumScore,ReScore,QReScore
AnsId,,,,,,,,,,,,,,,,,


In [63]:
votes_com_f[votes_com_f.CumScore>votes_com_f.CumVotes]

,,AnsId,Age,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,Upvotes,Downvotes,CumVotes,CumScore,ReScore,QReScore
AnsId,,,,,,,,,,,,,,,,,


In [64]:
#Accepted answers 687 and 990 didn't have acceptance dates registered?
votes_com_f[(votes_com_f['AcceptedAnsId'].notnull()) & (votes_com_f['AcceptedAge']<0)]

AnsId  Age  QuestionId  AnsWordCount  AcceptedAnsId  AcceptedAge  \
AnsId                                                                          
687   2075    687    0         686            31            687           -1   
      2076    687    4         686            31            687           -1   
      2077    687   35         686            31            687           -1   
      2078    687  435         686            31            687           -1   
688   2079    688    0         686            26            687           -1   
689   2080    689    0         686            48            687           -1   
      2081    689  256         686            48            687           -1   
690   2082    690    0         686            39            687           -1   
990   2715    990    0         987            62            990           -1   
      2716    990  121         987            62            990           -1   
2647  5708   2647  224         987            64            990           -1   
4708  8987   4708  748         987           152            990           -1   

            QScore  QVotes  Score  Votes  Upvotes  Downvotes  CumVotes  \
AnsId                                                                    
687   2075       0       0      5      5        5          0         5   
      2076       4       4      1      1        1          0         6   
      2077       4       4      1      1        1          0         7   
      2078       4       4      1      1        1          0         8   
688   2079       0       0      4      4        4          0         4   
689   2080       0       0      3      3        3          0         3   
      2081       4       4      1      1        1          0         4   
690   2082       0       0      1      1        1          0         1   
990   2715       0       0      1      1        1          0         1   
      2716       6       6      1      1        1          0         2   
2647  5708       6       6      2      2        2          0         2   
4708  8987       6       6     -1      1        0          1         1   

            CumScore   ReScore  QReScore  
AnsId                                     
687   2075         5  0.993820  0.000000  
      2076         6  0.994703  0.800000  
      2077         7  0.995365  0.800000  
      2078         8  0.995880  0.800000  
688   2079         4  0.992584  0.000000  
689   2080         3  0.990730  0.000000  
      2081         4  0.992584  0.800000  
690   2082         1  0.981460  0.000000  
990   2715         1  0.981460  0.000000  
      2716         2  0.987640  0.857143  
2647  5708         2  0.987640  0.857143  
4708  8987        -1 -0.018540  0.857143

## Calculate Rank

In [65]:
from itertools import izip
def rank_ans(df,score_only,re_score):
    rk_name = "ReScore_rank" if re_score else "AnsRank"
    def rank_iter():
        cache = {}
        accepted = 0
        for row in df.itertuples():
            if re_score:
                cache[row.AnsId] = row.ReScore
            else :
                cache[row.AnsId] = row.Score
            # rank, nb_ans
            if (not score_only) and row.AcceptedAge>-1 and (row.AnsId == row.AcceptedAnsId) and row.Age >=row.AcceptedAge:
                accepted = 1
                if row.AnsId in cache:
                    del cache[row.AnsId]
                yield (1,len(cache)+accepted,row.Index)
            else :
                rank = sorted(cache, key= lambda k:cache[k],reverse=True).index(row.AnsId) + 1 + accepted
                yield (rank,len(cache)+accepted,row.Index)
            
    ranks, ans_counts, indices = izip(*list(rank_iter())) #TODO: optimize for the future
    return [pd.Series(ranks,name=rk_name, index=indices), pd.Series(ans_counts,name="Ans_count", index=indices)]

predictors = ['QuestionId','AnsId','AnsWordCount','AcceptedAnsId','Age',
              'Score','Votes','Upvotes','Downvotes','CumScore','CumVotes','QScore'
              ,'QVotes','ReScore','QReScore','AnsRank','ReScore_rank']
get_ranks = lambda df,score_only=False,re_score=False: pd.concat(
    [df[cname] for cname in df.columns.values.tolist() if cname in predictors] + rank_ans(df,score_only,re_score),axis=1)
sort_age_score = lambda df: df.sort_values(by=['Age','Score'],ascending=[True,False])

## Base data for model

In [66]:
votes_com_f = votes_com_f.groupby(['QuestionId']).apply(
    lambda df: get_ranks(sort_age_score(df))).reset_index(drop=True)
votes_com_f = votes_com_f.groupby(['QuestionId']).apply(
    lambda df: get_ranks(sort_age_score(df),score_only=True,re_score=True)).reset_index(drop=True)

votes_com_f['Pbias'] = 1.0/votes_com_f['AnsRank']
votes_com_f['DRank'] = votes_com_f['AnsRank'] - votes_com_f['ReScore_rank']
votes_com_f.head()

,AnsId,Age,QuestionId,AnsWordCount,AcceptedAnsId,QScore,QVotes,Score,Votes,Upvotes,Downvotes,CumVotes,CumScore,ReScore,QReScore,AnsRank,ReScore_rank,Ans_count,Pbias,DRank
0,56,0,1,201,56,0,0,2,2,2,0,2,2,0.98764,0.000000,1,1,1,1.000000,0
1,4,0,1,190,56,0,0,1,3,2,1,3,1,0.49073,0.000000,2,2,2,0.500000,0
2,5,0,1,37,56,0,0,1,1,1,0,1,1,0.98146,0.000000,3,2,3,0.333333,1
3,56,1,1,201,56,12,12,3,3,3,0,5,5,0.99382,0.923077,1,1,3,1.000000,0
4,210,1,1,40,56,12,12,1,1,1,0,1,1,0.98146,0.923077,2,2,4,0.500000,0


In [68]:
#AnsRank and Ans_count define unique EPbias
sum_by_rank = lambda df: df.groupby('AnsRank').apply(
    lambda df: pd.Series([df.Votes.sum()],name='EPbias').to_frame()).unstack(level=-1).reset_index(level=0,drop=False)
get_ratio = lambda df: sum_by_rank(df).EPbias/(sum_by_rank(df).EPbias.sum())
ratio_per_rank = lambda df: pd.concat([sum_by_rank(df).AnsRank, get_ratio(df)],axis=1)
get_position_bias = lambda df: pd.merge(df,ratio_per_rank(df),how='inner',on=['AnsRank'])

votes = votes_com_f.groupby(['Ans_count']).apply(get_position_bias).reset_index(level=[0,1],drop=True)
votes.columns.values[-1] = "EPbias"
votes.head()

,AnsId,Age,QuestionId,AnsWordCount,AcceptedAnsId,QScore,QVotes,Score,Votes,Upvotes,...,CumVotes,CumScore,ReScore,QReScore,AnsRank,ReScore_rank,Ans_count,Pbias,DRank,EPbias
0,56,0,1,201,56,0,0,2,2,2,...,2,2,0.987640,0,1,1,1,1,0,1
1,10,0,2,145,10,0,0,3,3,3,...,3,3,0.990730,0,1,1,1,1,0,1
2,9,0,3,25,109,0,0,4,4,4,...,4,4,0.992584,0,1,1,1,1,0,1
3,16,0,7,45,2977,0,0,3,3,3,...,3,3,0.990730,0,1,1,1,1,0,1
4,20,0,14,28,20,0,0,7,7,7,...,7,7,0.995365,0,1,1,1,1,0,1


In [32]:
votes['EPbias'].value_counts()

1.000000    4900
0.546423    2551
0.453577    2111
0.370221    1033
0.374883     988
0.254896     695
0.328707     454
0.263722     378
0.218297     325
0.189274     253
0.305304     209
0.179819     136
0.186287     135
0.173351     130
0.155239     102
0.284153      91
0.193989      64
0.169399      58
0.416000      49
0.340136      45
0.117486      42
0.136612      40
0.299401      35
0.098361      33
0.108844      30
0.382979      30
0.149701      24
0.161677      23
0.085106      23
0.074830      21
0.137725      21
0.129252      18
0.107784      18
0.144000      18
0.180851      16
0.112000      14
0.095238      13
0.063830      12
0.048000      12
0.065868      11
0.068027      10
0.088000       9
0.077844       9
0.032000       8
0.056000       7
0.176471       6
0.352941       6
0.416667       5
0.166667       4
0.031915       3
0.024000       3
0.058824       3
0.083333       3
0.021277       2
0.117647       2
Name: EPbias, dtype: int64

In [69]:
test_epbias = votes.groupby(['Ans_count','AnsRank']).first().reset_index(
    level=[0,1],drop=False)[['Ans_count','AnsRank','EPbias']]
test_epbias.to_csv('EPbiasbyAnsCountRank.csv')

In [70]:
#GET NULL say yay!!!
tt = votes.groupby(['AnsId','Age']).Votes.count().reset_index(level=[0,1],drop=False)
tt[tt.Votes>2].head()

,AnsId,Age,Votes


In [71]:
votes.to_csv(path_or_buf='AnsVotes_TSeries.csv')

In [72]:
votes.columns

Index([u'AnsId', u'Age', u'QuestionId', u'AnsWordCount', u'AcceptedAnsId',
       u'QScore', u'QVotes', u'Score', u'Votes', u'Upvotes', u'Downvotes',
       u'CumVotes', u'CumScore', u'ReScore', u'QReScore', u'AnsRank',
       u'ReScore_rank', u'Ans_count', u'Pbias', u'DRank', u'EPbias'],
      dtype='object')